### Информация
- *Автор*: Киселев Кирилл
- *Дата*: вторник, 12 июля 2022 г.
- *Описание*: Блокнот для вычисления регионов для транспортных накладных

#### Requirements

In [2]:
# !pip3 install opencv-python==4.4.0.46 pandas==1.2.4

In [1]:
import cv2
import pandas as pd


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
data = pd.read_csv('/home/kirillk/datasets/OCR/ocr_ttn/test-cases-ttn.csv', index_col=0)

# data.loc[data['filename'] == '095.jpg', 'orientation'] = 'horizontal'

data.head()
# data.to_csv('../assets/test-cases.csv')

,filename,barcode_number,doc_number,barcode_box_left,barcode_box_top,barcode_box_width,barcode_box_height,doc_type_box_left,doc_type_box_top,doc_type_box_width,doc_type_box_height,doc_number_box_left,doc_number_box_top,doc_number_box_width,doc_number_box_height
0,001.jpg,2880107901381,0074786723/2053332022,0.036891,0.041373,0.182212,0.051974,0.167635,0.097458,0.085710,0.019087,0.361261,0.176741,0.103765,0.016444
1,002.jpg,2880108066928,0074801020/2053439854,0.034493,0.035611,0.182089,0.049772,0.167335,0.090945,0.084423,0.016688,0.359356,0.170287,0.104494,0.015810
2,003.jpg,2880108067130,0074801108/2053440150,0.044062,0.030233,0.181714,0.054957,0.174441,0.088082,0.086541,0.020247,0.366604,0.170679,0.104713,0.020247
3,004.jpg,2880108430682,0074837996/2053658941,0.045632,0.027665,0.183300,0.055418,0.176841,0.085854,0.085775,0.018565,0.370715,0.167041,0.103204,0.017457
4,005.jpg,2880108430712,0074837999/2053659636,0.035318,0.037956,0.187007,0.055973,0.169280,0.096473,0.087210,0.022050,0.363179,0.173648,0.107289,0.021202


## Исследование по вычислениям регионов

### Расположение штрих-кода

Отступ

In [20]:
barcode_indent = data['barcode_box_left'].min()
barcode_indent

0.0263062412307952

Ширина

In [21]:
data['barcode_box_left'].max()

0.0654671447284517

In [22]:
data['barcode_box_width'].max()

0.1870065263274597

In [23]:
barcode_region = (data['barcode_box_left'] + data['barcode_box_width']).max()
barcode_region

0.2387975539403071

In [24]:
barcode_width = (data['barcode_box_left'] + data['barcode_box_width']).max() - data['barcode_box_left'].min()
barcode_width

0.2124913127095119

In [11]:
# doc_type_region = (data['doc_type_box_left'] + data['doc_type_box_width']).max()
# doc_type_region

#### Визуализация результатов

In [12]:
for index, row in data.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_ttn/jpg/first_pages/' + row['filename'])
    image = cv2.resize(image, (0, 0), fx=0.3, fy=0.3)
    height, width, _ = image.shape
    
    x1 = int(row['barcode_box_left'] * width)
    y1 = int(row['barcode_box_top'] * height)
    x2 = int((row['barcode_box_left'] + row['barcode_box_width']) * width)
    y2 = int((row['barcode_box_top'] + row['barcode_box_height']) * height)
    
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    image = cv2.rectangle(img=image, 
                          pt1=(int(type_indent * width), 0),
                          pt2=(int(type_indent * width + doc_type_width * width), int(height)),
                          color=(0, 0, 255),
                          thickness=2)
    
    cv2.imshow('Document', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
#     if index == 2:
#         break

In [25]:
list_of_deltas = []

In [26]:
%%time
for index, row in data.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_ttn/jpg/first_pages/' + row['filename'])
    height, width, _ = image.shape
    
    x1 = int(row['barcode_box_left'] * width)
    x2 = int((row['barcode_box_left'] + row['barcode_box_width']) * width)
    
    list_of_deltas.append(barcode_region * width - x2)

CPU times: user 1.88 s, sys: 45.9 ms, total: 1.92 s
Wall time: 1.93 s


In [27]:
# list_of_deltas

In [28]:
max(list_of_deltas)

113.70181922259735

In [29]:
min(list_of_deltas)

0.9406167765375812

In [30]:
pd.Series(list_of_deltas).describe()

count     44.000000
mean      61.109209
std       26.503618
min        0.940617
25%       43.690617
50%       61.440617
75%       80.701819
max      113.701819
dtype: float64

**Вывод:** регион для поиска типа документа имеет значения

отступ по x = 0.0263

ширина региона = 0.2125

### Расположение типа документа

In [50]:
delta_x_type_array = (data['doc_type_box_left'] - data['barcode_box_width'] - data['barcode_box_left'])

delta_x_type = delta_x_type_array.describe()['min']
delta_x_type

-0.05304512044865439

In [51]:
delta_y_type_array = (data['doc_type_box_top'] - data['barcode_box_height'] - data['barcode_box_top'])

delta_y_type = delta_y_type_array.describe()['min'] 
delta_y_type

-0.0029132784640584983

In [52]:
height_type = delta_y_type_array.describe()['max'] - delta_y_type_array.describe()['min'] + data['doc_type_box_height'].describe()['max']
height_type


0.03209606854623259

In [65]:
width_type = delta_x_type_array.describe()['max'] - delta_x_type_array.describe()['min'] + data['doc_type_box_width'].describe()['mean']
width_type


0.09230489690081423

In [66]:
for index, row in data.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_ttn/jpg/first_pages/' + row['filename'])
    image = cv2.resize(image, (0, 0), fx=0.3, fy=0.3)
    height, width, _ = image.shape
    
    x1 = int(row['barcode_box_left'] * width)
    y1 = int(row['barcode_box_top'] * height)
    x2 = int((row['barcode_box_left'] + row['barcode_box_width']) * width)
    y2 = int((row['barcode_box_top'] + row['barcode_box_height']) * height)
        
    delta_x_type_ = int(delta_x_type * width)
    delta_y_type_ = int(delta_y_type * height)
    width_type_ = int(width_type * width)
    height_type_ = int(height_type * height)
    
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    image = cv2.rectangle(img=image,
                            pt1=(int(x2 + delta_x_type_), 
                                int(y2 + delta_y_type_)),
                            pt2=(int(x2 + delta_x_type_ + width_type_),
                                int(y2 + delta_y_type_ + height_type_)),
                            color=(0, 0, 255),
                            thickness=2)

    cv2.imshow('Document', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
#     if index == 7:
#         break

**Вывод:** принимаем следующие параметры региона **Типа документа**

дельта по x = -0.0530

дельта по y = -0.0030

ширина региона = 0.0924

высота региона = 0.0321

### Расположение номера документа

In [67]:
delta_x_number_array = (data['doc_number_box_left'] - data['barcode_box_width'] - data['barcode_box_left'])

delta_x_number = delta_x_number_array.describe()['min']
delta_x_number

0.1374539198685151

In [68]:
delta_y_number_array = (data['doc_number_box_top'] - data['barcode_box_height'] - data['barcode_box_top'])

delta_y_number = delta_y_number_array.describe()['min'] 
delta_y_number

0.061724535704068925

In [79]:
height_number = delta_y_number_array.describe()['max'] - delta_y_number_array.describe()['min'] + data['doc_number_box_height'].describe()['mean']
height_number

0.04528373702369032

In [80]:
width_number = delta_x_number_array.describe()['max'] - delta_x_number_array.describe()['min'] + data['doc_number_box_width'].describe()['mean']
width_number

0.13544483570610416

#### Визуализация результатов

In [82]:
for index, row in data.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_ttn/jpg/first_pages/' + row['filename'])
    image = cv2.resize(image, (0, 0), fx=0.3, fy=0.3)
    height, width, _ = image.shape
    
    x1 = int(row['barcode_box_left'] * width)
    y1 = int(row['barcode_box_top'] * height)
    x2 = int((row['barcode_box_left'] + row['barcode_box_width']) * width)
    y2 = int((row['barcode_box_top'] + row['barcode_box_height']) * height)
        
    delta_x_number_ = int(delta_x_number * width)
    delta_y_number_ = int(delta_y_number * height)
    width_number_ = int(width_number * width)
    height_number_ = int(height_number * width)
    
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    image = cv2.rectangle(img=image,
                            pt1=(int(x2 + delta_x_number_), 
                                int(y2 + delta_y_number_)),
                            pt2=(int(x2 + delta_x_number_ + width_number_),
                                int(y2 + delta_y_number_ + height_number_)),
                            color=(0, 0, 255),
                            thickness=2)

    cv2.imshow('Document', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
#     if index == 7:
#         break

**Вывод:** принимаем следующие параметры региона **Номер документа**

дельта по x = 0.1374

дельта по y = 0.0617

ширина региона = 0.1355

высота региона = 0.0453

 ### Расположение даты документа

#### Визуализация результатов

**Вывод:** принимаем следующие параметры региона **Дата документа**

дельта по x = 0.1158

дельта по y = -0.0121

ширина региона = 0.0878

высота региона = 0.0162

### Дефолтные значения для типа документа

In [83]:
doc_type_x1 = data['doc_type_box_left'].min()
doc_type_x1

0.1592948642586113

In [84]:
doc_type_y1 = data['doc_type_box_top'].min()
doc_type_y1

0.0682747230205484

In [90]:
doc_type_x2 = data['doc_type_box_left'].max() + data['doc_type_box_width'].mean()
doc_type_x2

0.27726365512379914

In [91]:
doc_type_y2 = data['doc_type_box_top'].max() + data['doc_type_box_height'].max()
doc_type_y2

0.1461591377225028

In [92]:
doc_type_default_width = doc_type_x2 - doc_type_x1
doc_type_default_width

0.11796879086518783

In [93]:
doc_type_default_height = doc_type_y2 - doc_type_y1
doc_type_default_height

0.0778844147019544

#### Визуализация результатов

In [95]:
for index, row in data.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_ttn/jpg/first_pages/' + row['filename'])
    image = cv2.resize(image, (0, 0), fx=0.3, fy=0.3)
    height, width, _ = image.shape
        
    x1 = int(doc_type_x1 * width)
    y1 = int(doc_type_y1 * height)
    x2 = int(doc_type_x2 * width)
    y2 = int(doc_type_y2 * height)
    
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 1)
    cv2.imshow('Document', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
#     if index == 7:
#         break

**Вывод:** принимаем следующие параметры дефолтного региона **Тип документа**

Левый верхний угол координаиа x = 0.1592

Левый верхний угол координата y = 0.0682

ширина региона = 0.1180

высота региона = 0.0779

### Дефолтные значения для номера документа

In [103]:
doc_number_x1 = data['doc_number_box_left'].min()
doc_number_x1

0.356974374500453

In [104]:
doc_number_y1 = data['doc_number_box_top'].min()
doc_number_y1

0.145090027327508

In [105]:
doc_number_x2 = data['doc_number_box_left'].max() + data['doc_number_box_width'].mean()
doc_number_x2

0.501781303489269

In [106]:
doc_number_y2 = data['doc_number_box_top'].max() + data['doc_number_box_height'].mean()
doc_number_y2

0.21665756093500205

In [107]:
doc_number_default_width = doc_number_x2 - doc_number_x1
doc_number_default_width

0.14480692898881597

In [108]:
doc_number_default_height = doc_number_y2 - doc_number_y1
doc_number_default_height

0.07156753360749404

#### Визуализация результатов

In [109]:
for index, row in data.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_ttn/jpg/first_pages/' + row['filename'])
    image = cv2.resize(image, (0, 0), fx=0.6, fy=0.6)
    height, width, _ = image.shape
        
    x1 = int(doc_number_x1 * width)
    y1 = int(doc_number_y1 * height)
    x2 = int(doc_number_x2 * width)
    y2 = int(doc_number_y2 * height)
    
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 1)
    cv2.imshow('Document', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    if index == 7:
        break

**Вывод:** принимаем следующие параметры дефолтного региона **Номер документа**

Левый верхний угол координаиа x = 0.3569

Левый верхний угол координата y = 0.1450

ширина региона = 0.1449

высота региона = 0.0716

### Дефолтные значения для даты документа

**Вывод:** принимаем следующие параметры дефолтного региона **Дата документа**

Левый верхний угол координаиа x = 0.2286

Левый верхний угол координата y = 0.0339

ширина региона = 0.1459

высота региона = 0.0505